In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
training_set = pd.read_csv('../input/train.csv')
testing_set = pd.read_csv('../input/test.csv')

In [3]:
training_set.info()
print()
testing_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [4]:
y_train = training_set['label']
x_train = training_set.drop(labels='label',axis=1)
del training_set

In [5]:
from keras.utils.np_utils import to_categorical    #Encoding the label
y_train = to_categorical(y_train, num_classes = 10)

Using TensorFlow backend.


In [6]:
x_train = x_train.values.reshape(-1,28,28,1)   #Reshaping the values
testing_set = testing_set.values.reshape(-1,28,28,1)

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=0.1)

In [8]:
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,Dropout

In [9]:
classifier = Sequential()

classifier.add(Convolution2D(filters = 32,kernel_size=(3,3),input_shape = (28,28,1),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.15))

classifier.add(Convolution2D(32,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.15))

classifier.add(Flatten())

classifier.add(Dense(output_dim = 128, activation='relu'))
classifier.add(Dense(output_dim=10,activation='softmax'))


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  


In [10]:
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
from keras.preprocessing.image import ImageDataGenerator   #data augmentation
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range=0.1,zoom_range=0.1,horizontal_flip=True)

x_test = x_test/255.0

In [12]:
train_datagen.fit(x_train)

In [13]:
#classifier.fit(x_train,y_train,epochs=20,batch_size=86)
classifier.fit_generator(train_datagen.flow(x_train,y_train,batch_size=32)
                         ,epochs=5,validation_data=(x_test,y_test),verbose=2,steps_per_epoch=x_train.shape[0] // 32)

Epoch 1/5
 - 23s - loss: 0.3671 - acc: 0.8833 - val_loss: 0.1322 - val_acc: 0.9574
Epoch 2/5
 - 14s - loss: 0.1368 - acc: 0.9567 - val_loss: 0.0867 - val_acc: 0.9719
Epoch 3/5
 - 14s - loss: 0.0985 - acc: 0.9690 - val_loss: 0.0634 - val_acc: 0.9807
Epoch 4/5
 - 14s - loss: 0.0793 - acc: 0.9741 - val_loss: 0.0799 - val_acc: 0.9752
Epoch 5/5
 - 14s - loss: 0.0714 - acc: 0.9775 - val_loss: 0.0598 - val_acc: 0.9810


In [14]:
from sklearn.metrics import confusion_matrix,classification_report
y_pred = classifier.predict(x_test)
Y_pred_classes = np.argmax(y_pred,axis = 1) 
Y_true = np.argmax(y_test,axis = 1) 
cm = confusion_matrix(Y_true, Y_pred_classes) 
cr = classification_report(Y_true,Y_pred_classes)

In [15]:
print(cr)
print()
print(cm)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       437
           1       1.00      0.99      0.99       441
           2       0.98      0.98      0.98       435
           3       0.99      0.98      0.98       423
           4       0.99      0.96      0.97       379
           5       0.98      0.97      0.98       378
           6       0.96      0.99      0.97       401
           7       0.98      0.99      0.98       467
           8       0.98      0.98      0.98       403
           9       0.96      0.97      0.97       436

    accuracy                           0.98      4200
   macro avg       0.98      0.98      0.98      4200
weighted avg       0.98      0.98      0.98      4200


[[435   0   0   0   0   1   1   0   0   0]
 [  0 436   3   0   0   0   1   1   0   0]
 [  1   1 425   1   0   3   3   0   1   0]
 [  0   0   0 416   0   1   1   1   3   1]
 [  0   0   0   0 362   0   2   2   0  13]
 [  0   0   0   3   0 368

In [16]:
res = classifier.predict(testing_set)

res = np.argmax(res,axis = 1)

res = pd.Series(res,name="Label")

In [17]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),res],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)